In [48]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.ops import unary_union

In [2]:
data = gpd.read_file('seattle_lu.geojson')

In [13]:
get_coords = lambda geometry: gpd.GeoSeries(geometry.centroid.coords[:][0])
coords = data[~data.geometry.isna()].geometry.apply(get_coords)
coords.columns = ['lon', 'lat']

In [14]:
data_coords = pd.concat([data, coords], axis=1)

In [17]:
# sanity check to make sure that things merged correctly
(~data_coords.loc[data_coords.geometry.isna(), ['lon', 'lat']].isna()).sum()

lon    0
lat    0
dtype: int64

In [18]:
seattle_limits = gpd.read_file('city_limits.geojson')

In [51]:
seattle_mask = data_coords.geometry.centroid.within(unary_union(seattle_limits.geometry))

In [63]:
seattle_data_coords = data_coords.loc[seattle_mask, :]
seattle_data_coords.to_file('seattle_lu.geojson')
seattle_data_coords.drop('geometry', axis=1).to_csv('seattle_lu.csv', index=False)